The main program that uses the database and defined python scripts to run the analysis

In [1]:
#This non-descript folder contains all the standalone python modules
from python import *

import pickle
import os
import numpy as np
import json
from ipyparallel import Client
from itertools import repeat

In [2]:
#Create data classes
#Year, Season
seasonsData = None
#RaceId, List of tuples of (driverId, constructorId, time)
qualiResultsData = None
#DriverId, name
driversData = None
#ConstructorId, name
constructorsData = None
#EngineId, name
enginesData = None

with open('seasonsData.txt', 'rb') as handle:
    seasonsData = pickle.load(handle)
    #print(seasonsData)
    
with open('qualiResultsData.txt', 'rb') as handle:
    qualiResultsData = pickle.load(handle)
    #print(qualiResultsData)
    
with open('driversData.txt', 'rb') as handle:
    driversData = pickle.load(handle)
    #print(driversData)
    
with open('constructorsData.txt', 'rb') as handle:
    constructorsData = pickle.load(handle)
    #print(constructorsData)
    
with open('enginesData.txt', 'rb') as handle:
    enginesData = pickle.load(handle)
    #print(enginesData)

In [3]:
scores = []
scores = list(map(runSimulation, range(40), repeat(seasonsData), repeat(qualiResultsData), repeat(driversData), 
                    repeat(constructorsData), repeat(enginesData)))
print(np.median(scores))

0.5672773279352226


So, now when the data has been analysed, it's time to use it to predict the next qualification's results! In this simplified version, the grand prix is specified by the parameter circuit.

In [4]:
(drivers, constructors, engines) = runSimReturnDrivConstEng(seasonsData, qualiResultsData, 
                                                            driversData, constructorsData, enginesData)

In [5]:
newDrivers = json.load(open('newDrivers.json'))["drivers"]
newDrivers = {int(did): cid for did, cid in newDrivers.items()}

#print(newDrivers)
driversToWrite = {}
for did, cid in newDrivers.items():
    driversToWrite[int(did)] = {}
    driversToWrite[int(did)]["name"] = drivers[int(did)].name
    driversToWrite[int(did)]["constructor"] = drivers[int(did)].constructor.name
with open('driversDescribe.json', 'w') as fp:
    json.dump(driversToWrite, fp)

In [6]:
circuit = 7

driverResults = {} # {did: {position: amount}}
for did, cid in newDrivers.items():
    driverResults[int(did)] = {}
for i in range(1000):
    scoreList = predictQualiResults(circuit, newDrivers, drivers, constructors, engines)
    for i, drivRes in enumerate(scoreList):
        if i not in driverResults[drivRes[0]]:
            driverResults[drivRes[0]][i] = 0
        driverResults[drivRes[0]][i] += 1
        
for did, res in driverResults.items():
    print("{0} ({1}):".format(drivers[int(did)].name, drivers[int(did)].constructor.name))
    for pos, amount in sorted(res.items(), key=lambda posAmount: posAmount[0]):
        print("\t{0}: {1} %".format(pos + 1, amount / 10.0))
        
with open('predictions.json', 'w') as fp:
    json.dump(driverResults, fp)

#for i, res in enumerate(scoreList):
    #print("{0}: {1} ({2})".format(i+1, drivers[res[0]].name, drivers[res[0]].constructor.name))        

Lewis Hamilton (Mercedes):
	1: 2.6 %
	2: 17.2 %
	3: 27.0 %
	4: 19.6 %
	5: 19.4 %
	6: 13.2 %
	7: 0.1 %
	8: 0.2 %
	10: 0.1 %
	15: 0.4 %
	20: 0.2 %
Fernando Alonso (McLaren):
	7: 0.1 %
	8: 0.2 %
	9: 2.4 %
	10: 11.5 %
	11: 25.9 %
	12: 26.7 %
	13: 20.7 %
	14: 9.7 %
	15: 1.7 %
	16: 0.9 %
	20: 0.2 %
Kimi Räikkönen (Ferrari):
	1: 32.6 %
	2: 39.1 %
	3: 10.8 %
	4: 7.9 %
	5: 6.0 %
	6: 3.3 %
	15: 0.3 %
Sebastian Vettel (Ferrari):
	1: 61.0 %
	2: 21.5 %
	3: 6.8 %
	4: 5.8 %
	5: 3.1 %
	6: 1.1 %
	10: 0.5 %
	15: 0.2 %
Romain Grosjean (Haas F1 Team):
	8: 0.1 %
	9: 1.0 %
	10: 7.4 %
	11: 20.9 %
	12: 28.4 %
	13: 26.0 %
	14: 13.6 %
	15: 1.2 %
	16: 1.0 %
	17: 0.2 %
	18: 0.1 %
	20: 0.1 %
Nico Hülkenberg (Renault):
	5: 0.1 %
	6: 3.8 %
	7: 65.2 %
	8: 14.3 %
	9: 9.7 %
	10: 5.1 %
	11: 0.7 %
	12: 0.8 %
	13: 0.1 %
	15: 0.1 %
	20: 0.1 %
Sergio Pérez (Force India):
	6: 0.3 %
	7: 9.3 %
	8: 31.1 %
	9: 26.0 %
	10: 20.7 %
	11: 5.2 %
	12: 3.4 %
	13: 2.5 %
	14: 0.7 %
	15: 0.5 %
	20: 0.3 %
Daniel Ricciardo (Red Bull):
	1: 1.